# 앙상블(Ensemble) 2 - AdaBoost, GBM

## 앙상블 학습 개요

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1]:
# GridSearchCV를 위한 사전 셋업
grid_learn = [0.0001, 0.001, 0.01, 0.03, 0.05, 0.1, 0.2, 0.25, 0.5, 1.0]
grid_n_estimator = [10, 50, 100, 300, 500]
grid_ratio = [0.1, 0.25, 0.5, 0.75, 1.0]
grid_max_features = [0.3, 0.5, 0.7, 1.0]
grid_max_depth = [1, 2, 4, 8]
grid_min_samples_leaf = [1, 2, 3, 10, 100, 1500]
grid_min_samples_split = [2, 4, 8, 16, 24, 30]
grid_seed = [0]

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42, max_depth=1)

ada_clf = AdaBoostClassifier(dt, n_estimators=200, \
    algorithm='SAMME.R', learning_rate=0.5)
ada_clf.fit(X_train, y_train)

# 부스팅:  GBM(Gradient Boosting Machine)

* 그래디언트 부스팅의 매개변수

In [3]:
from sklearn.ensemble import GradientBoostingClassifier

?GradientBoostingClassifier

Init signature:
GradientBoostingClassifier(
    *,
    loss='log_loss',
    learning_rate=0.1,
    n_estimators=100,
    subsample=1.0,
    criterion='friedman_mse',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_depth=3,
    min_impurity_decrease=0.0,
    init=None,
    random_state=None,
    max_features=None,
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    validation_fraction=0.1,
    n_iter_no_change=None,
    tol=0.0001,
    ccp_alpha=0.0,
)
Docstring:     
Gradient Boosting for classification.

This algorithm builds an additive model in a forward stage-wise fashion; it
allows for the optimization of arbitrary differentiable loss functions. In
each stage ``n_classes_`` regression trees are fit on the negative gradient
of the loss function, e.g. binary or multiclass log loss. Binary
classification is a special case where only a single regression tree is
induced.

:class:`sklearn.ensemble.HistGradientBoostingClassifier` i

# AdaBoost : 타이타닉


In [3]:
# 타이타닉 df를 adaboosting

In [8]:
df_titanic = pd.read_csv('titanic/df_titanic.csv')

In [11]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1043 entries, 0 to 1042
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  1043 non-null   int64  
 1   Age       1043 non-null   float64
 2   SibSp     1043 non-null   int64  
 3   Parch     1043 non-null   int64  
 4   Fare      1043 non-null   float64
 5   Pclass_2  1043 non-null   int64  
 6   Pclass_3  1043 non-null   int64  
 7   Sex_1     1043 non-null   int64  
dtypes: float64(2), int64(6)
memory usage: 65.3 KB


In [13]:
X = df_titanic.iloc[:, 1:]
X

,Age,SibSp,Parch,Fare,Pclass_2,Pclass_3,Sex_1
0,22.0,1,0,1.981001,0,1,1
1,38.0,1,0,4.266662,0,0,0
2,26.0,0,0,2.070022,0,1,0
3,35.0,1,0,3.972177,0,0,0
4,35.0,0,0,2.085672,0,1,1
...,...,...,...,...,...,...,...
1038,3.0,1,1,2.622855,0,1,0
1039,37.0,1,0,4.499810,0,0,0
1040,28.0,0,0,2.050913,0,1,0
1041,39.0,0,0,4.690430,0,0,0


In [19]:
y = df_titanic['Survived']
y

0       0
1       1
2       1
3       1
4       0
       ..
1038    1
1039    1
1040    1
1041    1
1042    0
Name: Survived, Length: 1043, dtype: int64

In [16]:
# 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# AdaBoost 모델 구축 및 학습, 평가

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state=42, max_depth=1)
ada_clf = AdaBoostClassifier(dt_clf, n_estimators=200, algorithm='SAMME.R', learning_rate=0.5)

ada_clf.fit(X_train, y_train)

train_score = accuracy_score(y_train, ada_clf.predict(X_train))
test_score = accuracy_score(y_test, ada_clf.predict(X_test))

print(train_score)
print(test_score)

0.86810551558753
0.8564593301435407


#### 하이퍼 파라미터
- 디폴트(default) 설정

```python
AdaBoostClassifier(
    base_estimator=None,
    n_estimators=50,
    learning_rate=1.0,
    algorithm='SAMME.R',
    random_state=None,
)
```
- base_estimator: 예측할 모델, 디폴트는 DecisionTreeClassifier(max_depth=1)
- n_estimators: 모형(week learner)의 갯수, 순차적으로 오류를 보정해서 수가 많으면 성능이 일정 수준까지 높아 질 수 있으나 수행 시간이 오래 걸린다는 단점이 있음. 디폴트는 50
- learning_rate: 학습률, 0~1 사이의 값을 지정. 너무 작은 값인 경우 최소점을 찾아 예측 성능이 높지만 학습에 오래 걸리고 너무 큰 값인 경우 최소점을 찾지 못해 예측 성능이 떨어질 확률이 높음. 그래서 n_estimators와 상호 호환 필요. 디폴트는 1.0

In [24]:
AdaBoostClassifier?

Init signature:
AdaBoostClassifier(
    estimator=None,
    *,
    n_estimators=50,
    learning_rate=1.0,
    algorithm='SAMME.R',
    random_state=None,
    base_estimator='deprecated',
)
Docstring:     
An AdaBoost classifier.

An AdaBoost [1] classifier is a meta-estimator that begins by fitting a
classifier on the original dataset and then fits additional copies of the
classifier on the same dataset but where the weights of incorrectly
classified instances are adjusted such that subsequent classifiers focus
more on difficult cases.

This class implements the algorithm known as AdaBoost-SAMME [2].

Read more in the :ref:`User Guide <adaboost>`.

.. versionadded:: 0.14

Parameters
----------
estimator : object, default=None
    The base estimator from which the boosted ensemble is built.
    Support for sample weighting is required, as well as proper
    ``classes_`` and ``n_classes_`` attributes. If ``None``, then
    the base estimator is :class:`~sklearn.tree.DecisionTreeClassifi

#### AdaBoost - GridSearchCV 를 활용한 최적의 파라미터 찾기

# ※ GradientBoost

# 1) 그래디언트 부스팅 with 타이타닉 Df

#### 하이퍼 파라미터
- 디폴트(default) 설정

```python
GradientBoostingClassifier(
    loss='deviance',
    learning_rate=0.1,
    n_estimators=100,
    subsample=1.0,
    criterion='friedman_mse',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_depth=3,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    init=None,
    random_state=None,
    max_features=None,
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    presort='auto',
    validation_fraction=0.1,
    n_iter_no_change=None,
    tol=0.0001,
)
```
- loss: 손실 함수(loss function), 디폴트는 'deviance'
- base_estimator: 예측할 모델, 디폴트는 DecisionTreeClassifier(max_depth=1)
- n_estimators: 모형(week learner)의 갯수, 순차적으로 오류를 보정해서 수가 많으면 성능이 일정 수준까지 높아 질 수 있으나 수행 시간이 오래 걸린다는 단점이 있음. 디폴트는 100
- learning_rate: 학습률, 0~1 사이의 값을 지정. 너무 작은 값인 경우 최소점을 찾아 예측 성능이 높지만 학습에 오래 걸리고 너무 큰 값인 경우 최소점을 찾지 못해 예측 성능이 떨어질 확률이 높음. 그래서 n_estimators와 상호 호환 필요. 디폴트는 0.1
- min_samples_leaf: 말단 리프 노드의 최소한의 샘플 데이터 수, 디폴트 1
- max_depth: 트리의 최대 깊이, 디폴트 3
- subsample: n_estimator 모형(week learner)이 학습에 사용하는 데이터의 샘플링 비율, 디폴트 1.0

#### GradientBoost - GridSearchCV 를 활용한 최적의 파라미터 찾기

-------------------------------------------------------------------------------------------------

# 2) 그래디언트 부스팅 with 사용자 행동 인식 데이터 세트

-----------------------------------------------------------------------------------

## 실습] 와인데이터셋 - 그래디언트 부스팅 분류 -> 점수->튜닝-> 점수-> 피처 시각화

# [실습] 대출 상환 데이터를 이용하여 결정 트리, 랜덤 포레스트, 에이다부스트, 그래디언트 부스트 모델을 차례로 구축하고 성능을 평가하라